# MASS: Howto Mini App Streaming Source (MASS)

This notebooks demonstrates the usage data source apps that can be used for the development and characterizing of streaming application.

For Light Source - see TomoPy manual: <https://tomopy.readthedocs.io/en/latest/ipynb/tomopy.html>

In [1]:
%%capture

%pylab inline
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import os, sys, time
sys.path.append("..")
import tomopy
import pandas as pd
import numpy
import ast
import dxchange
import tempfile
import pykafka
import base64
import io
import binascii
import tempfile
import pykafka
import datetime, time, json
import mass.kafka

## logging
import logging
logging.basicConfig(level=logging.ERROR)
logging.getLogger().setLevel(logging.ERROR)
logging.getLogger("py4j").setLevel(logging.ERROR)
logging.getLogger("radical.utils").setLevel(logging.ERROR)
 
import boto3
boto3.setup_default_session(profile_name='dev')    
    
# Pilot-Streaming
import pilot.streaming
sys.modules['pilot.streaming']

# Resource Setup

## Kafka

In [2]:
kafka_pilot_description1 = {
    "resource":"slurm+ssh://login1.wrangler.tacc.utexas.edu",
    "working_directory": os.path.join('/work/01131/tg804093/wrangler/', "work"),
    "number_cores": 48,
    "project": "TG-MCB090174",
    "queue": "normal",
    "walltime": 159,
    "type":"kafka"
}
kafka_pilot1 = pilot.streaming.PilotComputeService.create_pilot(kafka_pilot_description1)
kafka_pilot1.wait()
kafka_details = kafka_pilot1.get_details()
kafka_details

/tmp/tmpqfkg9ab4
Submission of Job Command: ssh login1.wrangler.tacc.utexas.edu sbatch  tmpqfkg9ab4
Cleanup: ssh login1.wrangler.tacc.utexas.edu rm tmpqfkg9ab4
**** Job: 105067 State : Queue
look for configs in: /work/01131/tg804093/wrangler/work/kafka-cf6a8a30-011f-11e9-b7d6-e7401968b77b/config
['broker-0']
Kafka Config: /work/01131/tg804093/wrangler/work/kafka-cf6a8a30-011f-11e9-b7d6-e7401968b77b/config (Sun Dec 16 04:46:46 2018)
{'broker.id': '0', 'listeners': 'PLAINTEXT://c251-101:9092', 'zookeeper.connect': 'c251-101:2181', 'zookeeper.connection.timeout.ms': '6000'}
look for configs in: /work/01131/tg804093/wrangler/work/kafka-cf6a8a30-011f-11e9-b7d6-e7401968b77b/config
['broker-0']
Kafka Config: /work/01131/tg804093/wrangler/work/kafka-cf6a8a30-011f-11e9-b7d6-e7401968b77b/config (Sun Dec 16 04:46:46 2018)
{'broker.id': '0', 'listeners': 'PLAINTEXT://c251-101:9092', 'zookeeper.connect': 'c251-101:2181', 'zookeeper.connection.timeout.ms': '6000'}


{'master_url': 'c251-101:2181',
 'details': {'broker.id': '0',
  'listeners': 'PLAINTEXT://c251-101:9092',
  'zookeeper.connect': 'c251-101:2181',
  'zookeeper.connection.timeout.ms': '6000'}}

## Kinesis

In [ ]:
number_partitions=4
pilot_compute_description = {
    "resource":"kinesis://awscloud.com",
    "number_cores": number_partitions,
    "type":"kinesis"
}
kinesis_pilot = pilot.streaming.PilotComputeService.create_pilot(pilot_compute_description)
kinesis_pilot.wait()
kinesis_details=kinesis_pilot.get_details()
kinesis_pilot.get_details()

## Dask

In [3]:
dask_pilot_description = {
    "resource":"slurm+ssh://login1.wrangler.tacc.utexas.edu",
    "working_directory": os.path.join('/work/01131/tg804093/wrangler/', "work"),
    "number_cores": 48,
    "project": "TG-MCB090174",
    "queue": "normal",
    "walltime": 159,
    "type":"dask"
}
dask_pilot = pilot.streaming.PilotComputeService.create_pilot(dask_pilot_description)
dask_pilot.wait()
dask_details = dask_pilot.get_details()
dask_details

/tmp/tmpzevpxicb
Submission of Job Command: ssh login1.wrangler.tacc.utexas.edu sbatch  tmpzevpxicb
Cleanup: ssh login1.wrangler.tacc.utexas.edu rm tmpzevpxicb
**** Job: 105069 State : Queue


{'master_url': 'tcp://c251-106:8786', 'web_ui_url': 'http://c251-106:8787'}

# Mini App Test

## KMeans

### Kinesis

In [2]:
miniapp=mass.kafka.MiniApp(
                            #dask_scheduler=dask_details['master_url'],
                            dask_scheduler=None,
                            resource_url=kinesis_details["master_url"],
                            broker_service="kinesis",
                            number_parallel_tasks=2,
                            number_clusters=10,
                            number_points_per_cluster=10000,
                            number_points_per_message=1000,
                            number_messages=1,
                            number_dim=3,
                            number_produces=1,
                            number_partitions=number_partitions,
                            topic_name="test",
                            application_type="kmeans"
                           )
miniapp.run()

NameError: name 'mass' is not defined

In [ ]:
boto3.setup_default_session(profile_name='dev')
kinesis_client = boto3.client('kinesis', region_name='us-east-1')
stream_name = kinesis_details["master_url"].split("/")[1]
print("Stream Name: %s"%stream_name)
stream = kinesis_client.describe_stream(StreamName=stream_name)['StreamDescription']

messages = []
for shard in stream['Shards']:
    print("### %s - %s"%(stream_name, shard['ShardId']))
    shard_iterator = kinesis_client.get_shard_iterator(
        StreamName=stream_name,
        ShardId=shard['ShardId'],
        ShardIteratorType='AT_TIMESTAMP',  #'TRIM_HORIZON'|'LATEST'
        Timestamp=datetime.datetime.utcnow() - datetime.timedelta(minutes=30)
    )['ShardIterator']

    out = kinesis_client.get_records(ShardIterator=shard_iterator, Limit=1000)
    if out["Records"]:
        for record in out["Records"]:
            #data = json.loads()
            messages.append(record["Data"])
    else:
        print(out)
        time.sleep(1)

In [ ]:
len(messages)

In [ ]:
count = 0
number_total_points = 0
for message in messages:
    data_np = np.array(ast.literal_eval(message.decode("utf-8")))
    num_points = data_np.shape[0]
    number_dimensions = data_np.shape[1]
    count =  count + 1
    number_total_points = number_total_points + num_points
        
print("Total Messages: %d, Total Points: %d, Number Dimensions: %d"%(count, number_total_points, number_dimensions))

### Kafka Broker

Ensure that the correct amount of data was successfully written to Kafka

In [2]:
%%time
miniapp=mass.kafka.MiniApp(
                            #dask_scheduler=dask_details['master_url'],
                            dask_scheduler='tcp://c251-101:8786',
                            #dask_scheduler=None,
                            resource_url='c251-119:2181',
                            #resource_url=kafka_details["master_url"],
                            broker_service="kafka",
                            number_parallel_tasks=1,
                            number_clusters=3,
                            number_points_per_cluster=1000,
                            number_points_per_message=1000,
                            number_messages=1,
                            number_dim=3,
                            number_produces=1,
                            number_partitions=1,
                            topic_name="test",
                            application_type="kmeans"
                           )
miniapp.run()

Number Messages: 3.0
MASS - Produce to Kafka
Use Dask Distributed
{'type': 'Scheduler', 'id': 'Scheduler-10a8c41a-2b5e-4fda-ac5e-a212dcbba37b', 'address': 'tcp://129.114.58.101:8786', 'services': {'bokeh': 8787}, 'workers': {'tcp://129.114.58.101:39136': {'type': 'Worker', 'id': 'tcp://129.114.58.101:39136', 'host': '129.114.58.101', 'resources': {}, 'local_directory': '/home/01131/tg804093/worker-dg07wtve', 'name': 'tcp://129.114.58.101:39136', 'ncores': 48, 'memory_limit': 134775570432, 'last_seen': 1544976832.9306831, 'services': {'nanny': 45920, 'bokeh': 46787}, 'metrics': {'cpu': 2.0, 'memory': 128180224, 'time': 1544976832.4297488, 'read_bytes': 5652.2798557586275, 'write_bytes': 6224.91607210541, 'num_fds': 28, 'executing': 0, 'in_memory': 0, 'ready': 0, 'in_flight': 0}}}}
Kafka/Kinesis: c251-119:2181, Dask: tcp://129.114.58.101:8786, Number Dask Nodes: 1,  Number Parallel Producers: 1
/home/01131/tg804093/work/kafka_2.11-2.1.0/bin/kafka-topics.sh --delete --zookeeper c251-119:2

In [4]:
%%time
miniapp=mass.kafka.MiniApp(
                            #dask_scheduler=dask_details['master_url'],
                            dask_scheduler='tcp://c251-101:8786',
                            #dask_scheduler=None,
                            resource_url='c251-119:2181',
                            #resource_url=kafka_details["master_url"],
                            broker_service="kafka",
                            number_parallel_tasks=4,
                            number_clusters=3,
                            number_points_per_cluster=1000,
                            number_points_per_message=1000,
                            number_messages=1,
                            number_dim=3,
                            number_produces=1,
                            number_partitions=1,
                            topic_name="test",
                            application_type="kmeans"
                           )
miniapp.run()

Number Messages: 3.0
MASS - Produce to Kafka
Use Dask Distributed
{'type': 'Scheduler', 'id': 'Scheduler-10a8c41a-2b5e-4fda-ac5e-a212dcbba37b', 'address': 'tcp://129.114.58.101:8786', 'services': {'bokeh': 8787}, 'workers': {'tcp://129.114.58.101:39136': {'type': 'Worker', 'id': 'tcp://129.114.58.101:39136', 'host': '129.114.58.101', 'resources': {}, 'local_directory': '/home/01131/tg804093/worker-dg07wtve', 'name': 'tcp://129.114.58.101:39136', 'ncores': 48, 'memory_limit': 134775570432, 'last_seen': 1544977106.931296, 'services': {'nanny': 45920, 'bokeh': 46787}, 'metrics': {'cpu': 0.0, 'memory': 142843904, 'time': 1544977106.4301481, 'read_bytes': 3496.481133312012, 'write_bytes': 4841.896686610459, 'num_fds': 28, 'executing': 0, 'in_memory': 0, 'ready': 0, 'in_flight': 0}}}}
Kafka/Kinesis: c251-119:2181, Dask: tcp://129.114.58.101:8786, Number Dask Nodes: 1,  Number Parallel Producers: 4
/home/01131/tg804093/work/kafka_2.11-2.1.0/bin/kafka-topics.sh --delete --zookeeper c251-119:21

In [ ]:
client = pykafka.KafkaClient(zookeeper_hosts=kafka_details["master_url"])
topic = client.topics['test']
consumer = topic.get_simple_consumer(reset_offset_on_start=True)
print("Number Brokers: "+str(len(client.brokers)))

In [ ]:
count = 0
number_total_points = 0
number_dimensions = 0
for i in range(100):
    message = consumer.consume(block=False)
    if message is not None:
        data_np = np.array(ast.literal_eval(message.value.decode("utf-8")))
        num_points = data_np.shape[0]
        number_dimensions = data_np.shape[1]
        count =  count + 1
        number_total_points = number_total_points + num_points
    #print "Consumed message: %d, Number Points: %d, Number Dimensions: %d"%\
    #        (count, num_points, number_dimensions)   
        
print("Total Messages: %d, Total Points: %d, Number Dimensions: %d"%(count, number_total_points, number_dimensions))

##  Light Source

This tests the light source reconstruction data production Mini App

Example GridRec Reconstruction

In [ ]:
import binascii
import tempfile
import pykafka
import dxchange
import tomopy

def reconstruct(message):
    start = 0
    end = 2
    #msg_bin = base64.urlsafe_b64decode(message.value)
    tf = tempfile.NamedTemporaryFile(delete=True)
    #tf = open("test.h5", "w")
    tf.write(message)
    tf.flush()
    proj, flat, dark, theta = dxchange.read_aps_32id(tf.name, sino=(start, end))
    theta = tomopy.angles(proj.shape[0])
    proj = tomopy.normalize(proj, flat, dark)
    rot_center = tomopy.find_center(proj, theta, init=290, ind=0, tol=0.5)
    proj = tomopy.minus_log(proj)
    recon = tomopy.recon(proj, theta, center=rot_center, algorithm='gridrec')
    recon = tomopy.circ_mask(recon, axis=0, ratio=0.95)
    return recon
    #tf.close()

### Kinesis

In [ ]:
%%time

pilot_compute_description = {
    "resource":"kinesis://awscloud.com",
    "number_cores": number_partitions,
    "type":"kinesis"
}
kinesis_light_pilot = pilot.streaming.PilotComputeService.create_pilot(pilot_compute_description)
kinesis_light_pilot.wait()
kinesis_light_details=kinesis_light_pilot.get_details()

miniapp=mass.kafka.MiniApp(
                           #dask_scheduler=dask_details['master_url'],
                           dask_scheduler=None,
                           resource_url=kinesis_light_details["master_url"],
                           broker_service="kinesis",
                           number_parallel_tasks=1,
                           number_messages=1,
                           number_produces=1,
                           number_partitions=number_partitions,
                           topic_name="light_test8",
                           application_type = "light"
                          )
miniapp.run()

In [ ]:
boto3.setup_default_session(profile_name='dev')
kinesis_client = boto3.client('kinesis', region_name='us-east-1')
stream_name = kinesis_light_details["master_url"].split("/")[1]
print("Stream Name: %s"%stream_name)
stream = kinesis_client.describe_stream(StreamName=stream_name)['StreamDescription']

messages = []
for shard in stream['Shards']:
    print("### %s - %s"%(stream_name, shard['ShardId']))
    shard_iterator = kinesis_client.get_shard_iterator(
        StreamName=stream_name,
        ShardId=shard['ShardId'],
        ShardIteratorType='AT_TIMESTAMP',  #'TRIM_HORIZON'|'LATEST'
        Timestamp=datetime.datetime.utcnow() - datetime.timedelta(minutes=30)
    )['ShardIterator']

    out = kinesis_client.get_records(ShardIterator=shard_iterator, Limit=10)
    if out["Records"]:
        for record in out["Records"]:
            #data = json.loads()
            messages.append(record["Data"])
    else:
        print(out)
        time.sleep(1)
print("Read %d messages from Kinesis"%len(messages))

In [ ]:
recon=reconstruct(messages[0])
plt.imshow(recon[0, :,:], cmap='Greys_r')
plt.show()

### Kafka

In [ ]:
%%time
topic_name = "light_test"
miniapp=mass.kafka.MiniApp(
                           #dask_scheduler=dask_details['master_url'],
                           dask_scheduler=None,
                           resource_url=kafka_details["master_url"],
                           broker_service="kafka",
                           number_parallel_tasks=1,
                           number_messages=2,
                           number_produces=2,
                           number_partitions=number_partitions,
                           topic_name=topic_name,
                           application_type = "light"
                          )

In [ ]:
print("Read from topic: %s"%topic_name)
client = pykafka.KafkaClient(zookeeper_hosts=kafka_details["master_url"])
topic = client.topics[topic_name]
consumer = topic.get_simple_consumer(reset_offset_on_start=True,fetch_message_max_bytes=10000000)
miniapp.run()
message = consumer.consume(block=False)

In [ ]:
type(message)

In [ ]:
%%time
recon=reconstruct(message.value)
plt.imshow(recon[0, :,:], cmap='Greys_r')
plt.show()

# Stop Pilots

In [33]:
kafka_pilot1.cancel()
dask_pilot.cancel()
kinesis_pilot.cancel()
kinesis_light_pilot.cancel()

**END**

---

---

***Scratch***

For testing serialization

In [ ]:
import pkg_resources, base64
import binascii
import tempfile

data = None
with open("../mass/tooth.h5", "r") as f:
    data = f.read()
    
data_enc=binascii.hexlify(data)
print data_enc[:20]
print type(data_enc)

tf = tempfile.NamedTemporaryFile(delete=True)
#tf = open("test.h5", "w")
tf.write(binascii.unhexlify(data_enc))
tf.flush()
proj, flat, dark, theta = dxchange.read_aps_32id(tf.name, sino=(0, 2))

In [ ]:
theta.dtype

In [ ]:
count = 0
number_total_points = 0
read_bytes = 0
for i in range(100):
    message = consumer.consume(block=False)
    if message is not None:
        print "Message %d, Bytes: %d"%(count, len(message.value))
        reconstruct(message)
        read_bytes = read_bytes + len(message.value)
        count =  count + 1
   
        
print("Total Messages: %d, Read Bytes: %d"%(count, read_bytes))

General Kafka Test

In [ ]:
producer.produce("hello")

In [ ]:
message = consumer.consume(block=False)
print message.value

In [ ]:
consumer.partitions

In [ ]:
from distributed import Client
dask_distributed_client = Client('tcp://c251-136:8786')

#def map_test():
#    return 1


class DaskTest():
    
    def __init__(self):
        self.dask_distributed_client = Client('tcp://c251-136:8786')


    def map_test(self):
        return 1
    
    def run(self):
        tasks = []
        for block_id in range(3):
            tasks.append(self.dask_distributed_client.submit(self.map_test))
           
        return self.dask_distributed_client.gather(tasks)
        

In [ ]:
t = DaskTest()
t.run()

In [ ]:
tasks = []
for block_id in range(3):
    tasks.append(dask_distributed_client.submit(map_test))
           
dask_distributed_client.gather(tasks)
#f = dask_distributed_client.submit(map_test)